In [1]:
import pandas as pd 
import numpy as np 
import torch 
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset

In [2]:
data = pd.read_csv('./data/diabetes.csv')
data.head()

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age,Class
0,6,148,72,35,0,33.6,50,positive
1,1,85,66,29,0,26.6,31,negative
2,8,183,64,0,0,23.3,32,positive
3,1,89,66,23,94,28.1,21,negative
4,0,137,40,35,168,43.1,33,positive


In [3]:
data.iloc[:,0:-1]

,Number of times pregnant,Plasma glucose concentration,Diastolic blood pressure,Triceps skin fold thickness,2-Hour serum insulin,Body mass index,Age
0,6,148,72,35,0,33.6,50
1,1,85,66,29,0,26.6,31
2,8,183,64,0,0,23.3,32
3,1,89,66,23,94,28.1,21
4,0,137,40,35,168,43.1,33
...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,63
764,2,122,70,27,0,36.8,27
765,5,121,72,23,112,26.2,30
766,1,126,60,0,0,30.1,47


In [4]:
x = data.iloc[:,0:-1].values
y_string = data.iloc[:,-1].values

In [5]:

y_int = []
for string in y_string:
    if string == "positive":
        y_int.append(1)
    else:
        y_int.append(0)

In [7]:
y = np.array(y_int,dtype='float64')

In [8]:
x

array([[  6. , 148. ,  72. , ...,   0. ,  33.6,  50. ],
       [  1. ,  85. ,  66. , ...,   0. ,  26.6,  31. ],
       [  8. , 183. ,  64. , ...,   0. ,  23.3,  32. ],
       ...,
       [  5. , 121. ,  72. , ..., 112. ,  26.2,  30. ],
       [  1. , 126. ,  60. , ...,   0. ,  30.1,  47. ],
       [  1. ,  93. ,  70. , ...,   0. ,  30.4,  23. ]])

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
sc = StandardScaler()
x = sc.fit_transform(x)
x

array([[ 0.63994726,  0.84832379,  0.14964075, ..., -0.69289057,
         0.20401277,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.69289057,
        -0.68442195, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -0.69289057,
        -1.10325546, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ...,  0.27959377,
        -0.73518964, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.69289057,
        -0.24020459,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.69289057,
        -0.20212881, -0.87137393]])

In [15]:
#we convert the array to pytorch tensor 
x = torch.tensor(x)
y = torch.tensor(y).unsqueeze(1)

C:\Users\danus\AppData\Local\Temp\ipykernel_25400\865281534.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x)
C:\Users\danus\AppData\Local\Temp\ipykernel_25400\865281534.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).unsqueeze(1)


In [16]:
x.shape,y.shape

(torch.Size([768, 7]), torch.Size([768, 1]))

In [32]:
class Dataset(Dataset):

    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __getitem__(self,index):
        # Get one item from the dataset
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)
    
    

In [33]:
dataset = Dataset(x,y)
len(dataset)
train_loader = torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=32,
                                           shuffle=True)

In [34]:


for (x,y) in train_loader:
    print("Data:{}".format(x.shape))
    print("Labels:{}".format(y.shape))

Data:torch.Size([32, 7])
Labels:torch.Size([32, 1])


In [35]:
# Now let's build the above network
class Model(nn.Module):
    def __init__(self, input_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features, 5)
        self.fc2 = nn.Linear(5, 4)
        self.fc3 = nn.Linear(4, 3)
        self.fc4 = nn.Linear(3, 1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [36]:
net = Model(x.shape[1])
net

Model(
  (fc1): Linear(in_features=7, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=3, bias=True)
  (fc4): Linear(in_features=3, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (tanh): Tanh()
)

In [39]:
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(net.parameters(),lr=0.01)

In [40]:
num_epochs = 200
for epoch in range(num_epochs):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        # Feed Forward
        output = net(inputs)
        # Loss Calculation
        loss = criterion(output, labels)
        # Clear the gradient buffer (we don't want to accumulate gradients)
        optimizer.zero_grad()
        # Backpropagation 
        loss.backward()
        # Weight Update: w <-- w - lr * gradient
        optimizer.step()
        
    #Accuracy
    # Since we are using a sigmoid, we will need to perform some thresholding
    output = (output>0.5).float()
    # Accuracy: (output == labels).float().sum() / output.shape[0]
    accuracy = (output == labels).float().mean()
    # Print statistics 
    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss, accuracy))

Epoch 1/200, Loss: 0.704, Accuracy: 0.438
Epoch 2/200, Loss: 0.704, Accuracy: 0.438
Epoch 3/200, Loss: 0.704, Accuracy: 0.438
Epoch 4/200, Loss: 0.703, Accuracy: 0.438
Epoch 5/200, Loss: 0.703, Accuracy: 0.438
Epoch 6/200, Loss: 0.703, Accuracy: 0.438
Epoch 7/200, Loss: 0.703, Accuracy: 0.438
Epoch 8/200, Loss: 0.703, Accuracy: 0.438
Epoch 9/200, Loss: 0.703, Accuracy: 0.438
Epoch 10/200, Loss: 0.702, Accuracy: 0.438
Epoch 11/200, Loss: 0.702, Accuracy: 0.438
Epoch 12/200, Loss: 0.702, Accuracy: 0.438
Epoch 13/200, Loss: 0.702, Accuracy: 0.438
Epoch 14/200, Loss: 0.702, Accuracy: 0.438
Epoch 15/200, Loss: 0.702, Accuracy: 0.438
Epoch 16/200, Loss: 0.701, Accuracy: 0.438
Epoch 17/200, Loss: 0.701, Accuracy: 0.438
Epoch 18/200, Loss: 0.701, Accuracy: 0.438
Epoch 19/200, Loss: 0.701, Accuracy: 0.438
Epoch 20/200, Loss: 0.701, Accuracy: 0.438
Epoch 21/200, Loss: 0.701, Accuracy: 0.438
Epoch 22/200, Loss: 0.701, Accuracy: 0.438
Epoch 23/200, Loss: 0.700, Accuracy: 0.438
Epoch 24/200, Loss: 

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sklearn.datasets 
import sklearn

In [44]:
x,y = sklearn.datasets.make_moons(200,noise=0.20)


In [45]:
y

array([0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       1, 1], dtype=int64)

In [ ]:
plt.sca